In [ ]:
#Summary of File: Test the Optimized DFC with the XgBoost and RF as the base layers instead of only the Random Forest
#All DFC layer combinations are tested with the AlexNet, ResNet, and MobileNet CNN combination for consistency

In [ ]:
#imports

from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Add, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import ReLU, AvgPool2D

#define the AlexNet CNN (acquired from AlexNet paper)
def create_cnn_alexnet():
    model = Sequential()
    model.add(Conv2D(filters = 96, input_shape = (224, 224, 3),  
                kernel_size = (11, 11), strides = (4, 4),  
                padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling  
    model.add(MaxPooling2D(pool_size = (2, 2), 
                strides = (2, 2), padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (11, 11),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 3rd Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 4th Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 5th Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # Flattening 
    model.add(Flatten()) 
    
    # 1st Dense Layer 
    model.add(Dense(4096, input_shape = (224*224*3, ))) 
    model.add(Activation('relu')) 
    # Add Dropout to prevent overfitting 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Dense Layer 
    model.add(Dense(4096)) 
    model.add(Activation('relu')) 
    # Add Dropout 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    model.add(Dense(2048, activation = 'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='relu'))
    return model

#define the ResNet CNN and its associated blocks (acquired from ResNet Paper)
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1',)(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')         (X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
#main ResNet Block
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', )(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    headModel = Flatten()(X)
    headModel=Dense(256, activation='relu', name='fc1')(headModel)
    headModel=Dense(128, activation='relu', name='fc2')(headModel)
    headModel = Dense( 1, name='fc3')(headModel)
    model = Model(inputs=X_input, outputs=headModel, name='ResNet50')
    return model

# Define MobileNet block (acquired from MobileNet paper)
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def create_cnn_mobilenet():
    input = Input(shape = (224,224,3))
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # main part of the model
    x = mobilnet_block(x, filters = 64, strides = 1)
    x = mobilnet_block(x, filters = 128, strides = 2)
    x = mobilnet_block(x, filters = 128, strides = 1)
    x = mobilnet_block(x, filters = 256, strides = 2)
    x = mobilnet_block(x, filters = 256, strides = 1)
    x = mobilnet_block(x, filters = 512, strides = 2)
    
    for _ in range (5):
        x = mobilnet_block(x, filters = 512, strides = 1)
    x = mobilnet_block(x, filters = 1024, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 1)
    x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
    x = Flatten()(x)
    x = Dense(units = 1024, activation = 'relu')(x)
    output = Dropout(0.4)(x)
    output = Dense (1)(output)
    model = Model(inputs=input, outputs=output)
    return model


gasdata = pd.read_csv('path to CSV file of gas data (including labels for training)')
data_dir = 'path to image files'
#load the image files
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        img = mpimg.imread("path to image")
        img.resize(224,224,3)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
#store loaded images

trainImagesX = images
trainAttrX = gasdata
#load the training labels
trainy = trainAttrX["Gas"]
#left with only gas data
trainAttrX = trainAttrX.drop(columns=['Gas'])
#normalize the gas data
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
#load the pretrained AlexNet model
alexnet = create_cnn_alexnet()
model.load_weights("path to pretrained weights")
alexnetPredict = alexnet.predict(trainImagesX)
print(alexnetPredict.shape)
#get AlexNet predictions for gas leak images

#load the pretrained ResNet model 
resnet = ResNet50()
model.load_weights("path to pretrained weights")
resnetPredict = resnet.predict(trainImagesX)
print(resnetPredict.shape)
#get ResNet predictions for gas leak images

#load pretrained MobileNet model
mobilenet = create_cnn_mobilenet()
model.load_weights("path to pretrained weights")
mobilenetPredict = mobilenet.predict(trainImagesX)
print(mobilenetPredict.shape)
#get MobileNet predictions

#finalize preparing the train data
alexnetPredict = np.array(alexnetPredict)
resnetPredict = np.array(resnetPredict)
mobilenetPredict = np.array(mobilenetPredict)
trainAttrX = np.array(trainAttrX)
trainAttrX.resize(5120,2)

trainData = concatenate([alexnetPredict,resnetPredict,mobilenetPredict,trainAttrX])
trainData = np.array(trainData)
trainData.reshape(-1,1)

print(trainData.shape)



In [9]:
from xgboost import XGBClassifier
from deepforest import CascadeForestClassifier  
from sklearn.ensemble import RandomForestClassifier

In [13]:
#load the base DFC model
model = CascadeForestClassifier()

#load the different base layers into the model per cascade layer
n_estimators = 2  # the number of base estimators per cascade layer
estimators = [XGBClassifier(n_estimators=100), XGBClassifier(n_estimators=100), RandomForestClassifier(random_state=3),RandomForestClassifier(random_state = 4)]
model.set_estimator(estimators)

In [ ]:
#train the model
model.fit(trainData, trainy)

In [ ]:
#Model Testing
gasdata = pd.read_csv('path to simulated testing data (gas data and annotations)')
#separate the gas data from the label
result = gasdata['Gas']
gasdata = gasdata.drop(columns=['Gas'])
#normalize the simulated testing gas data 
gasdata= (gasdata - np.min(gasdata)) / (np.max(gasdata) - np.min(gasdata))
#get image paths
images =  sorted(os.listdir("path to thermal images for simulated testing data folder"))
#load and read the testing data
X_dat = []
for image in images:
       img = mpimg.imread("path to image")
       #normalize the images 
       img.resize(224,224,3)
       img = img/255.0
       X_dat.append(img)
test_images = np.array(X_dat)

#use pretrained CNNs to get their predictions for the testing images
alexnetPredictTest = np.array(alexnet.predict(test_images))
resnetPredictTest = np.array(resnet.predict(test_images))
mobilenetPredictTest = np.array(mobilenet.predict(test_images))
print(alexnetPredictTest.shape)
print(resnetPredictTest.shape)
print(mobilenetPredictTest.shape)
print(gasdata.shape)

#finalize preparing the testing data
testData = concatenate([alexnetPredictTest, resnetPredictTest, mobilenetPredictTest, gasdata])
print(testData)
print(testData.shape)

#predict if there is gas leak or not on testing data
performance = model.predict(testData)
print(performance)
actual = []
for value in performance: 
    #print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)
acutal = np.array(actual)

#print the metrics for this model
accuracy = accuracy_score(result, actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(result, actual)
print('Precision: %f' % precision)
recall = recall_score(result, actual)
print('Recall: %f' % recall)
f1 = f1_score(result, actual)
print('F1 score: %f' % f1)